In [25]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination


## 1 - Gerar amostras

In [26]:
def carregar_dados_e_amostras(df_projeto, n_amostras=1000):
    """Lê os dados do projeto, que agora têm parâmetros individuais, e gera as amostras."""

    samples = {}
    print("Gerando amostras com distribuições individuais...")
    for _, row in df_projeto.iterrows():
        codigo = row['Código']
        params = [float(p) for p in row['Parâmetros'].split(',')]
        if row['Distribuição'] == 'triangular':
            samples[codigo] = np.random.triangular(left=params[0], mode=params[1], right=params[2], size=n_amostras)
    
    df_amostras = pd.DataFrame(samples)
    return df_projeto, df_amostras

## 2 - Discretizar as amostras

In [27]:
def discretizar_por_dias_inteiros(df_amostras):
    """
    Discretiza as amostras arredondando para o dia inteiro mais próximo.
    Os estados são os próprios dias. As probabilidades são suas frequências.
    """
    # 1. Arredonda todas as amostras para o inteiro mais próximo e converte para int
    df_arredondado = np.round(df_amostras).astype(int)
    
    parametros_discretizacao = {}
    print("\nDiscretizando por Dias Inteiros (arredondamento):")

    for atividade_codigo in df_arredondado.columns:
        # 2. Conta a frequência de cada dia inteiro para obter as probabilidades
        counts = df_arredondado[atividade_codigo].value_counts(normalize=True).sort_index()
        
        # 3. Os estados (labels) são os próprios dias inteiros
        estados = counts.index.tolist()
        probabilidades = counts.values.tolist()
        
        # O mapa de valor é o próprio estado (ex: o estado '3' tem valor 3)
        value_map = {estado: estado for estado in estados}
        
        parametros_discretizacao[atividade_codigo] = {
            'labels': estados,
            'value_map': value_map,
            'probs': probabilidades
        }
        # Este print mostrará o aumento da complexidade (número de estados)
        print(f" - Atividade {atividade_codigo}: {len(estados)} estados -> {estados}")
        
    return parametros_discretizacao

## 3 - Criar CPT para cada atividade

In [ ]:

def criar_cpt_termino(model, codigo, num_termino_states, termino_labels, params_discretizacao):
    t_node = f"T_{codigo}"
    d_node = f"D_{codigo}"
    
    activity_params = params_discretizacao[codigo]
    duration_labels = activity_params['labels']
    num_duration_states = len(duration_labels)

    parents = sorted(list(model.predecessors(t_node)))
    
    evidence = parents
    evidence_card = []
    state_names = {t_node: termino_labels}
    
    t_parents = [p for p in parents if p.startswith('T_')]
    
    for parent in parents:
        if parent.startswith('T_'):
            evidence_card.append(num_termino_states)
            state_names[parent] = termino_labels
        else: # parent.startswith('D_')
            parent_code = parent.split('_')[1]
            parent_labels = params_discretizacao[parent_code]['labels']
            evidence_card.append(len(parent_labels))
            state_names[parent] = parent_labels
            
    parent_state_combinations = itertools.product(*[range(card) for card in evidence_card])
    values = np.zeros((num_termino_states, np.prod(evidence_card)))
    
    for col_idx, combo in enumerate(parent_state_combinations):
        state_map = dict(zip(parents, combo))
        t_parent_states_indices = [state_map[p] for p in t_parents]
        max_of_t_parents = max(t_parent_states_indices) if t_parent_states_indices else -1
        
        d_state_index = state_map[d_node]
        d_state_value = params_discretizacao[codigo]['labels'][d_state_index] # Pega o valor real do dia
        
        # A lógica da soma agora usa valores de dias, não mais índices
        # Se T_A terminou na fase 5, e D_B dura 3 dias, T_B termina na fase 8
        result_state = min(max_of_t_parents + d_state_value, num_termino_states - 1)
        
        values[result_state, col_idx] = 1
        
    return TabularCPD(variable=t_node, variable_card=num_termino_states, values=values,
                      evidence=evidence, evidence_card=evidence_card, state_names=state_names)

## 4 - Construção da rede bayesiana

In [ ]:
def construir_rede_bayesiana_generica(df_projeto, params_discretizacao):
    G = nx.DiGraph()
    for _, row in df_projeto.iterrows():
        if row['Predecessoras'] != '-':
            for pred in row['Predecessoras'].split(','):
                G.add_edge(pred, row['Código'])
    
    # Estima o número de estados de término necessários (soma das durações máximas)
    max_duration_sum = 0
    if G.nodes:
        path = nx.dag_longest_path(G)
        max_duration_sum = sum(max(params['labels']) for code, params in params_discretizacao.items() if code in path)
    
    num_termino_states = max_duration_sum + 5
    termino_labels = list(range(num_termino_states)) 
    print(f"\nCriando {num_termino_states} estados de término (dias) para cobrir todas as possibilidades.\n")

    model = DiscreteBayesianNetwork()
    for _, row in df_projeto.iterrows():
        codigo = row['Código']
        model.add_edge(f"D_{codigo}", f"T_{codigo}")
        if row['Predecessoras'] != '-':
            for pred in row['Predecessoras'].split(','):
                model.add_edge(f"T_{pred}", f"T_{codigo}")

    all_cpds = []
    for _, row in df_projeto.iterrows():
        codigo = row['Código']
        params = params_discretizacao[codigo]
        d_node = f"D_{codigo}"
        prior_probs_array = np.array(params['probs']).reshape(len(params['labels']), 1)
        
        cpd_d = TabularCPD(variable=d_node, variable_card=len(params['labels']), 
                           values=prior_probs_array, state_names={d_node: params['labels']})
        all_cpds.append(cpd_d)
    
    # Adiciona as CPTs de Duração primeiro, para que a info esteja disponível
    model.add_cpds(*all_cpds)
    all_cpds = [] # Limpa a lista para adicionar as de término

    for _, row in df_projeto.iterrows():
        codigo = row['Código']
        cpd_t = criar_cpt_termino(model, codigo, num_termino_states, termino_labels, params_discretizacao)
        all_cpds.append(cpd_t)
        
    model.add_cpds(*all_cpds)

    print(f"Modelo Válido? {model.check_model()}\n")
    return model



## 5 - Main


In [ ]:


# Leitura da planilha
df = pd.read_excel("exemplo_aldo_dorea_pag170livro.xlsx", sheet_name="probabilistico")

# 1. Carregar dados e gerar amostras
df_projeto, df_amostras = carregar_dados_e_amostras(df)

# 2. Discretizar por DIAS INTEIROS
params_discretizacao = discretizar_por_dias_inteiros(df_amostras)

# 3. Construir a rede genérica
bn_model = construir_rede_bayesiana_generica(df_projeto, params_discretizacao)

# 4. Realizar Inferência
inference = VariableElimination(bn_model)

# Encontrar o nó final dinamicamente
G_final = nx.from_pandas_edgelist(df_projeto[df_projeto['Predecessoras'] != '-'].assign(Predecessoras=lambda x: x.Predecessoras.str.split(',')).explode('Predecessoras'), 
                                    source='Predecessoras', target='Código', create_using=nx.DiGraph)
final_node_code = [node for node in df_projeto['Código'] if G_final.out_degree(node, weight=None) == 0][0]
final_t_node = f"T_{final_node_code}"


# --- Análise e Cálculo do Valor Esperado ---
def calcular_valor_esperado(query_result):
    """Calcula a média ponderada a partir do resultado de uma query da pgmpy."""
    valor_esperado = 0
    # CORREÇÃO: Pegamos o nome da variável da lista .variables
    variable_name = query_result.variables[0] 
    
    for i, prob in enumerate(query_result.values):
        # O estado é o próprio valor em dias
        valor_estado = query_result.state_names[variable_name][i]
        valor_esperado += prob * valor_estado
    return valor_esperado

# Previsão INICIAL
query_inicial = inference.query(variables=[final_t_node], show_progress=False)
duracao_esperada_inicial = calcular_valor_esperado(query_inicial)

print(f"--- Previsão Inicial para o Término do Projeto ({final_t_node}) ---")
# print(query_inicial) # Descomente para ver a tabela de probabilidade completa
print(f"DURAÇÃO TOTAL ESPERADA INICIAL: {duracao_esperada_inicial:.2f} dias")


# Previsão ATUALIZADA
evidencia = {'D_A': 2, 'D_B': 4} # Evidência: A levou 1 dia, E levou 2 dias
query_atualizada = inference.query(variables=[final_t_node], evidence=evidencia, show_progress=False)
duracao_esperada_atualizada = calcular_valor_esperado(query_atualizada)

print(f"\n--- Previsão Atualizada (com evidência: {evidencia}) ---")
# print(query_atualizada) # Descomente para ver a tabela de probabilidade completa
print(f"DURAÇÃO TOTAL ESPERADA ATUALIZADA: {duracao_esperada_atualizada:.2f} dias")

Gerando amostras com distribuições individuais...

Discretizando por Dias Inteiros (arredondamento):
 - Atividade A: 2 estados -> [1, 2]
 - Atividade B: 3 estados -> [2, 3, 4]
 - Atividade C: 1 estados -> [1]
 - Atividade D: 3 estados -> [3, 4, 5]
 - Atividade E: 5 estados -> [2, 3, 4, 5, 6]
 - Atividade F: 3 estados -> [2, 3, 4]

Criando 20 estados de término (dias) para cobrir todas as possibilidades.

Modelo Válido? True

--- Previsão Inicial para o Término do Projeto (T_F) ---
DURAÇÃO TOTAL ESPERADA INICIAL: 9.76 dias

--- Previsão Atualizada (com evidência: {'D_A': 2, 'D_B': 3}) ---
DURAÇÃO TOTAL ESPERADA ATUALIZADA: 10.58 dias
